In [1]:
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt

In [2]:
def GetRGB(path):
    im_BGR = cv2.imread(path,cv2.COLOR_GRAY2RGB)
    im = cv2.cvtColor(im_BGR,cv2.COLOR_BGR2RGB)
    return im

# 设置为你的图片
# im = GetRGB('./3_2Photo/1.jpg')

In [1]:
# 尝试直方图均衡化， 
def CaculateHistogram(input_image):
    # 参数1. 单通道8位灰度图像
    # 输出1. 总像素值  
    # 输出2. 灰度值分布计数
    # 输出3. 灰度值分布前缀和（用来做后续直方图平局奴化）
    
    # 区分通道
    if len(np.shape(input_image)) == 3:
           height,width,level = np.shape(input_image)
           summ = height*width*level
    else :
           height,width = np.shape(input_image)
           summ = height*width

    
#     第一种 通过访问指定位置像素  耗时executed in 7m 9s
#     巨慢无比
#     for i in range(height):
#         for j in range(width):
#             caculate_num[input_image[i][j]] += 1
            
#     第二种 直接拆分行，像素获取像素值  耗时executed in 14.4s
#     for line in input_image:
#         for px in line:
#             caculate_num[px] +=1 

#     第三种 调用np.histogram()方法  耗时executed in 644ms
    caculate_num,index_x = GetHistogramArray(input_image)
    caculate_num = np.append(caculate_num,1)
#     print(np.shape(caculate_num))
    #     第四种 吕少推荐的方法(还没看懂)
#     caculate_num = cumsum(input_image,256)
#     print(np.shape(caculate_num))
    
    sum_num = np.copy(caculate_num)
    for i in range(1,256):
#         print(sum_num[i-1],caculate_num[i])
        sum_num[i] = sum_num[i-1] + sum_num[i]
    return summ,caculate_num,sum_num
    
def GetHistogramArray(image):
    return np.histogram(image,np.arange(0,256));

def cumsum(img, bins):
    
    histogram = np.zeros(bins)
    for pixel in np.arange(0, bins, 1):
        histogram[pixel] += len(img[img==pixel])
        
    return histogram
    
def HistogramEqualizationLUT(input_image):
    # 参数1, 单通道8位灰度图像
    # 输出1. 单通道8位灰度图像（直方图均衡化后）
    size,data,data_sum = CaculateHistogram(input_image)
    fxy  = lambda x: (255*data_sum[x])//size
    table = np.array([fxy(i) for i in range(256)])
#     print(table)
#     不可以这样
#     image_new = table[input_image]
    lut = lambda x: table[x]
    return lut(input_image),table


red_channel_he,red_lut_table = HistogramEqualizationLUT(im[:,:,0])
all_he,lut_table = HistogramEqualizationLUT(im)


plt.figure(figsize=(20,10))
ax1 = plt.subplot(231)
ax1.set_title("Original")
ax1.imshow(im)

ax2 = plt.subplot(232)
ax2.set_title("Red Channel HE")
ax2.imshow(red_channel_he, cmap = 'gray')


new_im = np.copy(im)
new_im[:,:,0] = red_channel_he
ax3 = plt.subplot(233)
ax3.set_title("Image With Red Channel HE")
ax3.imshow(new_im)

ax4 = plt.subplot(234)
ax4.set_title("Original")
ax4.imshow(im)

ax5 = plt.subplot(235)
ax5.set_title("All Channel Together HE")
ax5.imshow(all_he,cmap = "gray")

ax6 = plt.subplot(236)
ax6.set_title("All Channel Together HE")
ax6.imshow(all_he)

plt.show()


NameError: name 'im' is not defined

In [2]:
# OK, Do something interesting!
# 写一个整合函数，对每个输入图像的三通道都画出其三通道的函数变换图像和


def DrawPerChannel(input_img,split_channel : int, channel_color ,plotline = None, histogram = None, pic = None):
#     该通道的均衡曲线（分布函数曲线）
#     获取分离的通道
    input_channel = input_img[:,:,split_channel]
    temp_he,temp_table = HistogramEqualizationLUT(input_channel)
    if plotline != None:
        plotline.set_title("The function of "+channel_color+" channel")
        plotline.plot(np.arange(256),np.arange(256),color = "black",label = "$r_{" + channel_color +"}$")
        plotline.plot(np.arange(256),temp_table,color = channel_color ,label = "$T(r_{"+ channel_color +"})$")
        plotline.legend()
#     该通道的灰度分布直方图
    if histogram != None:
        kwargs = dict(bins = 25, histtype='bar', edgecolor = "white",alpha=0.5, density  = True)
        histogram.set_title("The histogram of "+channel_color+" channel")
        histogram.hist(input_channel.flatten(), color = "black",**kwargs, label = channel_color +" original")
        histogram.hist(temp_he.flatten(), color = channel_color , **kwargs, label = channel_color +" after he")
        histogram.legend()
#     该通道均匀后叠加到原图像的效果
    if pic != None:
        show_img = input_img.copy()
        show_img[:,:,split_channel] = temp_he
        pic.imshow(show_img)
    
    
    

def DrawChannelsEqualization(img_with_3channels):
    plt.figure(figsize=(30,30))
#     调用库函数
#     r,g,b = cv2.split(im)
#     直接分离  
#     r,g,b = img_with_3channels[:,:,0],img_with_3channels[:,:,1],img_with_3channels[:,:,2]
    DrawPerChannel(img_with_3channels,0,"red",plt.subplot(331),plt.subplot(334),plt.subplot(337))
    DrawPerChannel(img_with_3channels,1,"green",plt.subplot(332),plt.subplot(335),plt.subplot(338))
    DrawPerChannel(img_with_3channels,2,"blue",plt.subplot(333),plt.subplot(336),plt.subplot(339))
    plt.show()

DrawChannelsEqualization(im)

NameError: name 'im' is not defined

In [3]:
# 测试三通道分别做均衡化后效果和采取统一函数做均衡化的效果
def DrawCompareEqualization(im_with_3_channel):
    together_img,together_table = HistogramEqualizationLUT(im_with_3_channel)
    seperate_img = im_with_3_channel.copy()
    seperate_table = np.array([together_table,together_table,together_table])
    for i in range(3):
        seperate_img[:,:,i],seperate_table[i] = HistogramEqualizationLUT(im_with_3_channel[:,:,i])
    
    plt.figure(figsize=(15,10))
    plotline0 = plt.subplot(231)
    plotline0.set_title("No Change")
    plotline0.plot(np.arange(256),np.arange(256),color = "black",label = "$r$")
    plotline1 = plt.subplot(232)
    plotline1.set_title("Seperate Change")
    plotline1.plot(np.arange(256),np.arange(256),color = "black",label = "$r$")
    plotline1.plot(np.arange(256),seperate_table[0],color = "red" ,label = "$T(r_{red})$")   
    plotline1.plot(np.arange(256),seperate_table[1],color = "green" ,label = "$T(r_{green})$")
    plotline1.plot(np.arange(256),seperate_table[2],color = "blue" ,label = "$T(r_{blue})$")
    plotline1.legend()
    plotline2 = plt.subplot(233)
    plotline2.set_title("Together Change")
    plotline2.plot(np.arange(256),np.arange(256),color = "black",label = "$r$")
    plotline2.plot(np.arange(256),together_table,color = "orange" ,label = "$T(r_{together})$")
    plotline2.legend()
    pic0 = plt.subplot(234)
    pic0.imshow(im_with_3_channel)
    pic1 = plt.subplot(235)
    pic1.imshow(seperate_img)
    pic2 = plt.subplot(236)
    pic2.imshow(together_img)
    plt.show()

DrawCompareEqualization(im)

NameError: name 'im' is not defined

In [4]:
# 多来几组样例

imtest1 = imageio.imread('imageio:chelsea.png')
imtest2 = GetRGB('./3_2Photo/2.jpg')
imtest3 = GetRGB('./3_2Photo/3.jpg')
imtest4 = GetRGB('./3_2Photo/4.tif')

DrawCompareEqualization(imtest1)
# DrawCompareEqualization(imtest2)
DrawCompareEqualization(imtest3)
DrawCompareEqualization(imtest4)

NameError: name 'imageio' is not defined

In [5]:
# 展示直方图
def DrawHist(input_img,pic_handle,histogram_handle):
    kwargs = dict(bins = 25, histtype='bar', edgecolor = "white",alpha=0.5, density  = True)
    pic_handle.set_title("The Image")
    pic_handle.imshow(input_img,cmap = "gray")
    histogram_handle.set_title("The histogram_handle")
    histogram_handle.hist(input_img.flatten(),**kwargs)


plt.figure(figsize=(20,15))
DrawHist(imtest1[:,:,0],plt.subplot(321),plt.subplot(322))
DrawHist(imtest2[:,:,0],plt.subplot(323),plt.subplot(324))
DrawHist(imtest3[:,:,0],plt.subplot(325),plt.subplot(326))
plt.show()

NameError: name 'plt' is not defined